#Load in Files

###Imports

In [2]:
spark.version

u'2.2.0'

In [62]:
import re
import datetime

#import org.apache.commons.io.IOUtils
#import java.net.URL
#import java.nio.charset.Charset
#import com.databricks.spark.csv
#import org.apache.spark.sql.SQLContext
#import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType,DoubleType,DateType,TimestampType};
#import scala.util.matching.Regex
#import org.apache.spark.mllib.regression.LabeledPoint
#import org.apache.spark.mllib.linalg.Vectors
#import org.apache.spark.ml.classification.LogisticRegression
#import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
#import org.apache.spark.sql._
#import org.apache.spark.sql.types.{StructType,StructField,StringType,DoubleType};
#import org.apache.spark.sql.DataFrame
#import org.apache.spark.sql.Column
from pyspark.sql.functions import *
#// Need this for my shorthand $ notation
#import sqlContext.implicits._  


import pandas as pd
pd.set_option('display.max_colwidth', 80)

### Add Lookup Tables

In [63]:
monthMap = {
    "Jan": "01",
    "Feb": "02",
    "Mar": "03",
    "Apr": "04",
    "May": "05",
    "Jun": "06",
    "Jul": "07",
    "Aug": "08",
    "Sep": "09",
    "Oct": "10",
    "Nov": "11",
    "Dec": "12"
}

teamMap = {
  "Atlanta" : "atl",
  "Boston"  : "bos",
  "Brooklyn"  : "bkn",
  "Charlotte"  : "cha",
  "Chicago"  : "chi",
  "Cleveland"  : "cle",
  "Dallas"  : "dal",
  "Denver"  : "den",
  "Detroit"  : "det",
  "Golden State"  : "gst",
  "Houston"  : "hou",
  "Indiana"  : "ind",
  "LA Clippers"  : "lac",
  "LA Lakers"  : "lal",
  "Memphis"  : "mem",
  "Miami"  : "mia",
  "Milwaukee"  : "mil",
  "Minnesota"  : "min",
  "New Orleans"  : "nor",
  "New York"  : "nyk",
  "Oklahoma City"  : "okc",
  "Orlando"  : "orl",
  "Philadelphia"  : "phi",
  "Phila."  : "phi",
  "Phoenix"  : "pho",
  "Portland"  : "por",
  "Sacramento" : "sac",
  "San Antonio"  : "san",
  "Toronto"  : "tor",
  "Utah"  : "uta",
  "Washington"  : "wsh",
   None : "none"}

### Load In NBA Score Data Set

In [64]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,DateType

customSchema = StructType([
    StructField("dateOrig", DateType()),
    StructField("ts", StringType()),
    StructField("teamlonga", StringType()),
    StructField("scorea", IntegerType()),
    StructField("teamlongb", StringType()),
    StructField("scoreb", IntegerType()),
    StructField("timestring", StringType()),
    StructField("timeleft", DoubleType()),
    StructField("gameid", IntegerType())
])

nbafile = '/data2/nba-rt-prediction/scoredata/scores_nba.2015.test.dat'
rtscoresAndFinalDF = spark.read.format('csv')\
                    .option("header", "false")\
                    .option("inferSchema", "false")\
                    .option("nullValue", "empty")\
                    .option("dateFormat", "yyyy-MM-dd")\
                    .option("mode","DROPMALFORMED")\
                    .schema(customSchema)\
                    .load(nbafile).coalesce(2)
    
rtscoresAndFinalDF.show(5)


+----------+--------+-------------+------+------------+------+------------+--------+---------+
|  dateOrig|      ts|    teamlonga|scorea|   teamlongb|scoreb|  timestring|timeleft|   gameid|
+----------+--------+-------------+------+------------+------+------------+--------+---------+
|2016-04-05|15:06:16|      Phoenix|     0|     Atlanta|     0|(8:00 PM ET)|    48.0|400829044|
|2016-04-05|15:06:16|      Chicago|     0|     Memphis|     0|(8:00 PM ET)|    48.0|400829045|
|2016-04-05|15:06:16|    Cleveland|     0|   Milwaukee|     0|(8:00 PM ET)|    48.0|400829046|
|2016-04-05|15:06:16|Oklahoma City|     0|      Denver|     0|(9:00 PM ET)|    48.0|400829047|
|2016-04-05|15:06:16|  New Orleans|     0|Philadelphia|     0|(7:00 PM ET)|    48.0|400829041|
+----------+--------+-------------+------+------------+------+------------+--------+---------+
only showing top 5 rows



In [65]:
rtscoresAndFinalDF.withColumn("blah", dayofmonth(rtscoresAndFinalDF["dateOrig"])).show()

+----------+--------+-------------+------+------------+------+-------------+--------+---------+----+
|  dateOrig|      ts|    teamlonga|scorea|   teamlongb|scoreb|   timestring|timeleft|   gameid|blah|
+----------+--------+-------------+------+------------+------+-------------+--------+---------+----+
|2016-04-05|15:06:16|      Phoenix|     0|     Atlanta|     0| (8:00 PM ET)|    48.0|400829044|   5|
|2016-04-05|15:06:16|      Chicago|     0|     Memphis|     0| (8:00 PM ET)|    48.0|400829045|   5|
|2016-04-05|15:06:16|    Cleveland|     0|   Milwaukee|     0| (8:00 PM ET)|    48.0|400829046|   5|
|2016-04-05|15:06:16|Oklahoma City|     0|      Denver|     0| (9:00 PM ET)|    48.0|400829047|   5|
|2016-04-05|15:06:16|  New Orleans|     0|Philadelphia|     0| (7:00 PM ET)|    48.0|400829041|   5|
|2016-04-05|15:06:16|      Detroit|     0|       Miami|     0| (8:00 PM ET)|    48.0|400829042|   5|
|2016-04-05|15:06:16|    Charlotte|     0|     Toronto|     0| (7:30 PM ET)|    48.0|400829

In [66]:
rtscoresAndFinalDF.filter(rtscoresAndFinalDF.timestring.contains("FINAL")).show(5)
rtscoresAndFinalDF.filter(rtscoresAndFinalDF.timestring.contains("1ST")).show(5)

+----------+--------+-----------+------+------------+------+----------+--------+---------+
|  dateOrig|      ts|  teamlonga|scorea|   teamlongb|scoreb|timestring|timeleft|   gameid|
+----------+--------+-----------+------+------------+------+----------+--------+---------+
|2016-04-05|21:22:09|New Orleans|    93|Philadelphia|   107|   (FINAL)|     0.0|400829041|
|2016-04-05|22:08:42|  Charlotte|    90|     Toronto|    96|   (FINAL)|     0.0|400829043|
|2016-04-05|22:25:25|    Chicago|    92|     Memphis|   108|   (FINAL)|     0.0|400829045|
|2016-04-05|22:28:58|    Phoenix|    90|     Atlanta|   103|   (FINAL)|     0.0|400829044|
|2016-04-05|22:30:29|  Cleveland|   109|   Milwaukee|    80|   (FINAL)|     0.0|400829046|
+----------+--------+-----------+------+------------+------+----------+--------+---------+
only showing top 5 rows

+----------+--------+-----------+------+------------+------+-------------+-------------+---------+
|  dateOrig|      ts|  teamlonga|scorea|   teamlongb|scor

### UDFs For Creating Extra Columns In Real Time Data Frame

In [67]:
# Create new team name column.. do simple lookup conversion with a UDF
def mapper(teamin) :
    return teamMap[teamin]

mapperudf = udf(mapper)


In [73]:
# Date Logic to adjust for games that finish on the day after .... 
# This is so that I can join them against the spread which was dated the day prior...
# This is due to not having a great key to join my tables ...

datecrossregex = re.compile("^0[0-3]") # midnight to 3am
def dateadjust(datein, tsin ) : 
    #dateary = datein.split("-")
    tsary   = tsin.split(":")
    sub_one_day = datetime.timedelta(days=1)
    newdate = datein
    if datecrossregex.match(tsary[0]) :
        #day = "%02d".format(int(dateary[2]) -1)
        #newdate = dateary(0) + "-" + dateary(1) + "-" + day   
        newdate = datein - sub_one_day
    return str(newdate)

dateadjustudf = udf(dateadjust)


# UDFs to create some extra features ... this one is for an experiemental combination of Time left and Score difference.  
# Made this via intuition.  This can be extended to add other custom features
import math
def scoredivtimeXform(numerator, denominator):
    rv = numerator/(math.pow(denominator+1,0.5))
    return rv
scoredivtimeUdf = udf(scoredivtimeXform)


### Wrangle The Real Time And Final Score Data.  Add Columns To The Data Set

In [74]:
# Remove Overtime games from this analysis
rtscoresAndFinalDF = rtscoresAndFinalDF.filter(~col("timestring").like("%OT%"))
# Create short 3 character team names 
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("teama", mapperudf(col("teamlonga")))
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("teamb", mapperudf(col("teamlongb")))



In [75]:
# Add a score differential Column 
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("scorea-scoreb", col("scorea") - col("scoreb"))

# Transform the Date.  This is for games that spanned multiple days and gave me a headache.  
# Games adjusted to the day they started on.
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("date",  dateadjustudf(col("dateOrig"),col("ts")))

# Create a Key for me to use to join with my odds data later.  Key = date.teama.teamb
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("key", concat(col("date"),lit("."),col("teama"),lit("."),col("teamb")))


In [77]:
rtscoresAndFinalDF.show()


+----------+--------+-------------+------+------------+------+-------------+--------+---------+-----+-----+-------------+----------+------------------+
|  dateOrig|      ts|    teamlonga|scorea|   teamlongb|scoreb|   timestring|timeleft|   gameid|teama|teamb|scorea-scoreb|      date|               key|
+----------+--------+-------------+------+------------+------+-------------+--------+---------+-----+-----+-------------+----------+------------------+
|2016-04-05|15:06:16|      Phoenix|     0|     Atlanta|     0| (8:00 PM ET)|    48.0|400829044|  pho|  atl|            0|2016-04-05|2016-04-05.pho.atl|
|2016-04-05|15:06:16|      Chicago|     0|     Memphis|     0| (8:00 PM ET)|    48.0|400829045|  chi|  mem|            0|2016-04-05|2016-04-05.chi.mem|
|2016-04-05|15:06:16|    Cleveland|     0|   Milwaukee|     0| (8:00 PM ET)|    48.0|400829046|  cle|  mil|            0|2016-04-05|2016-04-05.cle.mil|
|2016-04-05|15:06:16|Oklahoma City|     0|      Denver|     0| (9:00 PM ET)|    48.0|400

### Separate The Real Time And Final Data From One Common Dataframe To Two Dataframes

In [ ]:
# Currently based on the way the data was sampled, both real time scores and final scores are written as seperate records to the same file.  I need to pull these apart, and then join the dataframes so that I have a real time score and features, and know if the game was won or lost ....

# Create Final Score DF
# Note a shortcut for repeating the dataframe within the filter is to use a $   df.filter(df("foo").contains ... is equiv to df.filter($"foo".contains)

var finalscoresDF = rtscoresAndFinalDF.filter($"time-string".like("%FINAL%"))

# Rename some columns so that join later doesnt have name overlaps
finalscoresDF = finalscoresDF.withColumnRenamed("scorea", "fscorea")
finalscoresDF = finalscoresDF.withColumnRenamed("scoreb", "fscoreb")

// Create final score difference
finalscoresDF = finalscoresDF.withColumn("fscorea-fscoreb", $"fscorea" - $"fscoreb")

// Add a Win/loss column Win = 1, Loss = 0
finalscoresDF = finalscoresDF.withColumn("win-loss-enc", ($"fscorea-fscoreb" > 0).cast("double"))

// Remove Halftime records as this particular case isn't handled well... (for now)
var rtscoresDF = rtscoresAndFinalDF.filter(!$"time-string".like("%FINAL%")).filter(!$"time-string".like("HALFTIME"))

// Create final score difference
rtscoresDF = rtscoresDF.withColumn("scorea-scoreb", $"scorea" - $"scoreb")

// Create a unique feature based on my custom UDF.  Idea here is that I have intuition that timeleft and score difference are a strong predictor when combined
rtscoresDF = rtscoresDF.withColumn("score-div-time", scoredivtimeUdf($"scorea" - $"scoreb", $"timeleft"*2))

### Lets Take A Look Of What We Have For The Two Dataframes We Just Wrangled

In [ ]:
// Some Printouts .....
println("final scores data frame")
finalscoresDF.show(5)
println("real time scores data frame")
rtscoresDF.show(5)

### Interpret the Odds data
````How to interpret the odds data ...
Example Golden State -12.5 O (207.0) -125.0 | Detroit 12.5 U (207.0) 145.0
Here Golden State is a 12.5 pt favorite to win.  The over under is in parentheses (207) and is the 50/50 line between teams sum of scores
being above/below that line.  
Finally the -125 / +145 numbers are whats known at the moneyline odds. 
    A negative number means you need to bet 125$ to get a 100$ payout
    A positive number means you need to bet 100$ to get a 145$ payout
```

### Load In Odds Data

In [ ]:
# Here, the data is very raw, and needs to be pre-processed .  I will start by loading it as an RDD and perform a lot of transformations.  Once I have it properly parsed, I will convert to a dataframe.
# This is not beautiful, but gets the job done
# Data format .....
#       <title>New Orleans 2.5 O (207.0) 125.0 | Phila. -2.5 U (207.0) -145.0 (Apr 05, 2016 07:10 PM)</title>
#       <title>Detroit 4.0 O (202.0) 160.0 | Miami -4.0 U (202.0) -190.0 (Apr 05, 2016 08:05 PM)</title>

// Reading the data in as an RDD first.  There isn't a dataframe parser for this XML I have, so I will write a custom parser ....
val oddsrdd = sc.textFile(oddsfile)

// just grabbing the text within the < ... > tags.  I can do this, because the format is super simple and not nested
val gameStringRdd = oddsrdd.map(x => x.substring(x.indexOf('>')+1,x.lastIndexOf('<')))

// String to Double converter helper
def parseDouble(s: String) = try { Some(s.toDouble) } catch { case _ : Throwable => None }

// This is where I do the heavy lifting of parsing my XML .. and then finally convert my RDD to a dataframe .....
// just lots of string parsing and data type conversions
val oddsDF = gameStringRdd.map(x => {
  // find the period, and then find the space prior ot the period
  // Philadelphia special case.  since later on I index on a decimal point, I am assuming its part of a number and not a team abbrev
  // also removing commas with the cryptic filterNot section ... essentially 
   val x1 = x.replace("Phila.", "Philadelphia").filterNot("," contains _)
   val ss1 = x1.substring(0,x1.indexOf('.'))
   val teamlonga = ss1.substring(0,ss1.lastIndexOf(' '))
   val teama = teamMap(teamlonga)
   val ss2 = x1.substring(ss1.lastIndexOf(' ')+1,x1.length)
   val teamaspread = parseDouble(ss2.substring(0,ss2.indexOf(' ')) )
   val ss3 = ss2.substring(ss2.indexOf(' ')+1,ss2.length)
   val overunder = parseDouble(ss3.substring(ss3.indexOf('(')+1,ss3.indexOf(')')))
   val ss4 = ss3.substring(ss3.indexOf(')')+2,ss3.length)
   val teamaml = parseDouble(ss4.substring(0,ss4.indexOf(' ')))
   val ss5x = ss4.substring(ss4.indexOf('|')+2,ss4.length)
   val ss5 = ss5x.substring(0,ss5x.indexOf('.'))
   
   val teamlongb = ss5.substring(0,ss5.lastIndexOf(' '))
   val teamb = teamMap(teamlongb)
   val ss6 = ss5x.substring(ss5.lastIndexOf(' ')+1,ss5x.length)

   val teambml = parseDouble(ss6.substring(ss6.indexOf(')')+2,ss6.lastIndexOf('(')-1))
   val ss7 = ss6.substring(ss6.lastIndexOf('(')+1,ss6.length)
   val dateInfo = ss7.split(' ')
   val dateStr = dateInfo(2) + "-" + monthMap(dateInfo(0)) + "-" + dateInfo(1)
   // This will become my join key for the other data sets
   val key = dateStr +"." + teama + "." + teamb
  (key,teamlonga,teama,teamaspread,overunder,teamaml,teamlongb,teamb,teambml,dateStr)
   
  }
).toDF("key","teamlonga","teama","teamaspread","overunder","teamaml","teamlongb","teamb","teambml","dateStr")




###Inspect Some Of The Odds Data

In [ ]:
oddsDF.show(5)
printf("Total Home Teams      = %2d\n",oddsDF.select("teama").distinct.sort("teama").count())
printf("Total Away Teams      = %2d\n",oddsDF.select("teamb").distinct.sort("teamb").count())
printf("Total Games Collected = %d\n ",oddsDF.count())


###Join The Odds And Final Score Data Sets

In [ ]:
// Here is where we join the Odds/Realtime scores/ Final Scores into one wholistic data set as input for Logistic Machine Learning

// Create a smaller Final Score Dataframe.  Just keep the key, final score a and b, the win/loss indicator
var finalslicedscoresDF = finalscoresDF.select($"key",$"fscorea",$"fscoreb",$"win-loss-enc")

// First Join the 2 smallest data frames ... odd and final.
var gameDF = oddsDF.join(finalslicedscoresDF, oddsDF("key") === finalscoresDF("key")).drop(oddsDF("key"))
// Drop these redundant columns prior to joining with Realtime score dataframe
gameDF = gameDF.drop("teamlonga")
gameDF = gameDF.drop("teamlongb")
gameDF = gameDF.drop("teama")
gameDF = gameDF.drop("teamb")

// Print Out the Game Dataframe ... notice we have the odds data merged with the win loss data ....
println("gameDF")
gameDF.show(3)
printf("Total Joined Games Collected = %d\n ",gameDF.count())



###Join The Game Dataframe With The Real Time Score Dataframe

In [ ]:
val lrDF = rtscoresDF.join(gameDF, rtscoresDF("key") === gameDF("key")).drop(gameDF("key"))
println("lrDF : Logistic Regression Data Frame")
lrDF.show(3)
printf("Total Data Points in DataSet = %d\n ",lrDF.count())



###Lets Look At Some Stats From 'Unpacked' Logistic Regression Dataframe

In [ ]:
 lrDF.describe().show()

###'Pack' Logistic Dataframe Into Required Format For Logistic Regression [R] - Creating A Simple/Complex Dataframe For Comparison

In [ ]:
// Logistic regression requires that
// The best way I found to modify data in a custom way is with map, however it returns an RDD, so you will need to run toDF at the end
// Logistic DF requires a DF of type  => [label: double, features: vector]

import org.apache.spark.sql._
// These is a helper function that converts an 'Any(Int)' type to a Double or Any(Double) to a Double
val ai2d : (Any => Double) = (in:Any) => in.asInstanceOf[java.lang.Integer].doubleValue
val ad2d : (Any => Double) = (in:Any) => in.asInstanceOf[java.lang.Double]

var nbaSimpleLrDF = lrDF.map {
    dfrow => {
    // get the indexes ... prob should move this out of the looop!
      val tgt = ad2d( dfrow(dfrow.fieldIndex("win-loss-enc")))
      val f1  = ai2d( dfrow(dfrow.fieldIndex("scorea-scoreb")))
      val f2 =  ad2d( dfrow(dfrow.fieldIndex("timeleft")))
      LabeledPoint(tgt,Vectors.dense(f1,f2))
    }
}.toDF("label","features")

var nbaComplexLrDF = lrDF.map {
    dfrow => {
    // get the indexes ... prob should move this out of the looop!
      val tgt = ad2d( dfrow(dfrow.fieldIndex("win-loss-enc")))
      val f1  = ai2d( dfrow(dfrow.fieldIndex("scorea-scoreb")))
      val f2  = ad2d( dfrow(dfrow.fieldIndex("teamaspread")))
      val f3  = ad2d( dfrow(dfrow.fieldIndex("overunder")))
      val f4  = ad2d( dfrow(dfrow.fieldIndex("teamaml")))
      val f5 =  ad2d( dfrow(dfrow.fieldIndex("timeleft")))
      // Add a new feature that weight point differential more as time left gets smaller...
      // I played around with excel and this looked ok ....
      val f6  = ad2d( dfrow(dfrow.fieldIndex("score-div-time")))
      LabeledPoint(tgt,Vectors.dense(f1,f2,f3,f4,f5,f6))
    }
}.toDF("label","features")


nbaSimpleLrDF.show(3)
nbaComplexLrDF.show(3)

### Function to Create The Model And Train It And Test It 

In [ ]:
def trainAndTest( indf : org.apache.spark.sql.DataFrame, modelPath : String ) : (org.apache.spark.sql.DataFrame, org.apache.spark.ml.classification.LogisticRegressionModel)  = {
    val splits = indf.randomSplit(Array(0.60,0.39,0.01), seed = 11L)
    val trainingdf = splits(0).cache()
    val testdf = splits(1).cache()
    val crossvaldf = splits(2).cache()
    
    println("Tranining Samples = " + trainingdf.count())
    println("Test      Samples = " + testdf.count())
    println("Cross Val Samples = " + crossvaldf.count())
    
    // Setup some of the configurations for the Logistic regression model ..
    // Here we could try a pipeline with params to select the 'best setting' but in the interest of time will go with this
    val lr = new LogisticRegression()
      .setMaxIter(50)
    .setRegParam(0.10)
    .setElasticNetParam(0.0)

    // Fit the model
    val lrModel   = lr.fit(trainingdf)

    println("Reg Parameter:    =" + lrModel.getRegParam)
    println("lrModel.intercept = " + lrModel.intercept)
    println("lrModel.weights   = " + lrModel.weights)

    // Save the model for later use ....
    // Argh ! -> in 1.6.1 api, but not 1.5.2 :(  
    // lrModel.save("modelPath" )
    
    ////  Create a logistic regression summary object ////
    // val lrSummary = lrModel.summary
    // println("lrSummary.objectiveHistory = " + lrSummary.objectiveHistory.length)
    // println(lrSummary.objectiveHistory.deep.mkString("\n"))
    ////
    
    // transform is now used in lieu of predict from mllib.  Found this after studying the API for a while
    val predictions = lrModel.transform(testdf)

    // Select (prediction, true label) and compute test error
    val evaluator = new MulticlassClassificationEvaluator()
      .setLabelCol("label")
      .setPredictionCol("prediction")
      .setMetricName("precision")
    
    val accuracy = evaluator.evaluate(predictions)
    println("Test Error = " + (1.0 - accuracy))

    // return the 
    (predictions,lrModel)
}

### Test And Train Multiple Models

In [ ]:
def time[R](block: => R): R = {
  val t0 = System.nanoTime()
  val result = block
  println("Elapsed time: " + (System.nanoTime - t0) + "ns")
  result
 }
 
val (simplePredictionDF, simpleModel)   = trainAndTest(nbaSimpleLrDF, "/data/resources/nbaSimpleModel")
val (complexPredictionDF, complexModel) = trainAndTest(nbaComplexLrDF, "/data/resources/nbaComplexModel")


In [ ]:
complexPredictionDF.show(5)

### Helper To Unpack MLLIB Label and Features Vector Into A Standard Dataframe

In [ ]:
// The whole point to doing this is so that I can visualize the features and the outcomes.  Zeppelin cannot render the Vector type well at all, so converting the data back to a flattened style of layout
// Expect a column name to be passed that is the features vector, all other columns should come back as-is
// I could extend the MLLIB DF to do this !! and  do it



 // Create a Row from values.
// Row(value1, value2, value3, ...)
 // Create a Row from a Seq of values.
// Row.fromSeq(Seq(value1, value2, ...))

// Goal of this function to return a new flattened data frame with the Vector datatype replaced with multiple columns.  This
// is done so that I can use zeppelin to graph my data since it doesnt really handle vectors well for graphing.
// not super flexible as of now, but someday might make it better ...

def convertVectorDF (indf : org.apache.spark.sql.DataFrame, labelIndex : Int, featureIndex : Int, probIndex : Int, predictionIndex : Int) = {

    //indf.select(colname).show(5)
    val myCols = indf.columns
    
    // debug stuff
    println("featureIndex value = " + myCols(featureIndex))
    println("probIndex value = " + myCols(probIndex))
    println("predictionIndex value = " + myCols(predictionIndex))
    
    if(myCols(labelIndex) != "label") {
        println("labelIndex value = " + myCols(labelIndex))
        println("Error")
    }
    
    
    println(" myCols.length = " + myCols.length)
    //println("Vector Size = " + indf[0].features)
    indf.show(3)
    
    // build an RDD of sql Rows here, then use toDF to convert back to a Dataframe..
    val tmprdd = indf.map { line => 
        var reind = 0
        var newRow : Seq[Double]= Seq(line(labelIndex).asInstanceOf[Double])
        var labeledPointVector = line(featureIndex).asInstanceOf[org.apache.spark.mllib.linalg.Vector]
        for(i <- 0 to labeledPointVector.size -1 ) {
            newRow = newRow :+ labeledPointVector(i).asInstanceOf[Double]
        }
        var probArray = line(probIndex).asInstanceOf[org.apache.spark.mllib.linalg.Vector]
        newRow = newRow :+ probArray(0).asInstanceOf[Double]
        newRow = newRow :+ line(predictionIndex).asInstanceOf[Double]
        //  https://spark.apache.org/docs/1.4.0/api/java/org/apache/spark/sql/Row.html see example where 
        //  Row.fromSeq is used to make a Row
        Row.fromSeq(newRow)
    }
    //http://spark.apache.org/docs/latest/sql-programming-guide.html#programmatically-specifying-the-schema
    //val schema =
      // StructType(
      // schemaString.split(" ").map(fieldName => StructField(fieldName, StringType, true)))
    //tmprdd
    //https://spark.apache.org/docs/1.6.1/api/java/org/apache/spark/sql/types/StructType.html

    // Take the first row. Each row just has a single 'Row' object at position 0
    val numCols = tmprdd.take(1)(0).length
   //val struct =
    //StructType(
     //StructField("label", IntegerType, true) ::
     //StructField("f1", LongType, false) ::
     //StructField("c", BooleanType, false) :: Nil)
     //sqlContext.createDataFrame(tmprdd,schema)
    var schemaString = myCols(labelIndex);
    for(i <-0 to numCols - 4){
        schemaString = schemaString + ",f" +i 
    }
    schemaString = schemaString + "," + myCols(probIndex);
    schemaString = schemaString + "," + myCols(predictionIndex);
    
    //schemaString = schemaString + tmprdd.take(1)(0).toString
    println(schemaString)
    val schema  = StructType(schemaString.split(",").map(fieldName => StructField(fieldName,DoubleType,true)))
    val finalDF = sqlContext.createDataFrame(tmprdd, schema)
    finalDF
}


###Lets Publish The Model and Analysis And Vizualize In Jupyter ....

In [ ]:
var simpleUnpackedPredictionDF = convertVectorDF(simplePredictionDF,0,1,3,4)
simpleUnpackedPredictionDF = simpleUnpackedPredictionDF.withColumn("correct", when($"label" === $"prediction",1).otherwise(0))

//var complexUnpackedPredictionDF = convertVectorDF(complexPredictionDF,0,1,3,4)
//complexPredictionDF = complexPredictionDF.withColumn("correct", when($"label" === $"prediction",1).otherwise(0))

simpleUnpackedPredictionDF.write.format("json").save("swift://notebooks.spark/test/simple001.json")




In [ ]:
simpleUnpackedPredictionDF.count()

In [ ]:
%AddJar https://github.com/dustinvanstee/random-public-files/raw/master/SystemML.jar
import org.apache.sysml.api.MLContext

In [ ]:
import org.apache.sysml.api.MLContext